In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

from collections import defaultdict, OrderedDict, Counter
from copy import deepcopy
import time
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy.stats import poisson
from datetime import date, datetime
from common.databases.mdb import Reader
import uuid
import os
import ujson
import requests
from PIL import Image
import io
import tensorflow as tf
import pickle

from statoil import cfg
from statoil.utils import json_load, print_info, pickle_load


Populating the interactive namespace from numpy and matplotlib


# See the data

In [2]:
traindev = pickle_load(cfg.DATASETS["traindev_short"])

In [3]:
example = traindev[0]

for key in example.keys():
    print_info(example[key], key)

inc_angle: val:34.4718, type:<class 'float'>
is_iceberg: val:1, type:<class 'int'>
id: val:20b1e634, type:<class 'str'>
band_1: type:<class 'numpy.ndarray'>, shape:(75, 75), dtype:float32, min:-28.76497459411621, max:7.201947212219238
synthetic: val:0, type:<class 'int'>
synthetic_method: val:None, type:<class 'NoneType'>
band_2: type:<class 'numpy.ndarray'>, shape:(75, 75), dtype:float32, min:-34.419029235839844, max:-3.9169700145721436


# Other checks

## Check dataset distint

In [4]:
train = pickle_load(cfg.DATASETS["train"])
dev = pickle_load(cfg.DATASETS["dev"])
traindev = pickle_load(cfg.DATASETS["traindev"])

train_ids = {example["id"] for example in train}
dev_ids = {example["id"] for example in dev}
traindev_ids = {example["id"] for example in traindev}

In [5]:
def print_data_stats(dataset, name):
    synthetic = sum([example.get("synthetic", 0) for example in dataset])
    print("{}: len:{}, orig_examples: {}({:.1f}%), synthetic: {}({:.1f}%)".format(
            name, 
            len(dataset), 
            len(dataset) - synthetic, 
            (len(dataset) - synthetic)/len(dataset)*100,
            synthetic, 
            synthetic/len(dataset)*100,
        ))

print_data_stats(train, "train")
print_data_stats(dev, "dev")
print_data_stats(traindev, "traindev")

train: len:10264, orig_examples: 1283(12.5%), synthetic: 8981(87.5%)
dev: len:2568, orig_examples: 321(12.5%), synthetic: 2247(87.5%)
traindev: len:12832, orig_examples: 1604(12.5%), synthetic: 11228(87.5%)


In [6]:
print("len(dev) = {}".format(len(dev)))
print("len(traindev) = {}".format(len(traindev)))

print(len(train_ids & dev_ids))
print(len(traindev_ids & dev_ids))
print(len(traindev_ids & train_ids))



len(dev) = 2568
len(traindev) = 12832
0
2568
10264
